In [ ]:
%matplotlib notebook
%load_ext rpy2.ipython
from IPython.display import Image
import graphviz
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import pandas as pd
from pyvis.network import Network
import pydotplus
import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri
pandas2ri.activate()
readRDS = robjects.r['readRDS']


def get_num_edges(nodes):
    """Returns number of edges in a complete graph given number of nodes"""
    return nodes * (nodes - 1)/2

def get_num_nodes(edges):
    """Returns number of nodes in a complete graph given number of edges"""
    return 0.5 * (1 + np.sqrt(1 + 8*edges))

# Number of nodes
Plot to see what is the upper bound on the number of edges given number of nodes. Useful to see the range of edges for benchmarks.

In [ ]:
num = np.arange(10, 1000, 10)
fig, ax = plt.subplots()
ax.plot(num, get_num_edges(num))
ax.set_title("Complete graph")
ax.set_xlabel("Number of nodes")
ax.set_ylabel("Number of edges")
ax.ticklabel_format(style='sci', axis='y', scilimits=(0,0))

# Visualising networks
There are two ways: with `networkx` module and `pyvis` module.

## networkx
`networkx` gives static plots: 

In [ ]:
fname = "Output/Erdos/E300_N100_I10_M10_S1_P2_2/cplex/network_solution.dot"

g = nx.drawing.nx_agraph.read_dot(fname)
pos = nx.spring_layout(g, 0.1, iterations=50)

fig, ax = plt.subplots()
nx.draw(g, pos, ax, with_labels=True)

## Pyvis
`pyvis` gives interactive plots:

In [ ]:
g = Network(500, 1000, notebook=True)
g.from_DOT(fname)
g.show(fname.replace(".dot", ".html"))

The method shown below, of going through networkx, allows edges and nodes to be accessible through `g.get_edges()` and `g.get_nodes()` methods

In [ ]:
g = Network(1000, 1000, bgcolor="#414141", notebook=True)
g.from_nx(nx.drawing.nx_agraph.read_dot(fname))
# g.show_buttons()
g.set_options("""
var options = {
  "nodes": {
    "font": {
      "color": "rgba(153,153,153,1)"
    }
  },
  "edges": {
    "color": {
      "inherit": true
    },
    "smooth": false
  },
  "physics": {
    "minVelocity": 0.75
  }
}
""")
g.show(fname.replace(".dot", "2.html"))

## Graphviz

In [ ]:
with open(fname) as fh:
    dot_graph = fh.read()
graphviz.Source(dot_graph)

## pydotplus

In [ ]:
with open(fname) as fh:
    dot_graph = fh.read()
pydot_graph = pydotplus.graph_from_dot_data(dot_graph)
Image(pydot_graph.create_png(), width=400)

# Running R code
Running R code in python jupyter notebook

In [ ]:
%%R
print(readRDS("Output/Erdos/E100_N20_I3_M3_S1_P2_2/cplex/result.Rds")$weightedSIF)
print(readRDS("Output/Erdos/E100_N20_I3_M3_S1_P2_2/cbc/result.Rds")$weightedSIF)
print(readRDS("Output/Erdos/E100_N20_I3_M3_S1_P2_2/lpSolve/result.Rds")$weightedS)